<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Lab 02 - TRỰC QUAN HOÁ DỮ LIỆU 📌
</div>

<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Giai đoạn 3 - Data Modelling 📌
</div>

### Nhập các thư viện cần thiết

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter('ignore')

### Đọc dữ liệu

In [2]:
df = pd.read_csv('./data/Water_Cleaner_2.csv', parse_dates=['Date'], date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
df.head()

,Date,Depth_to_Groundwater,Temperature,Drainage_Volume,River_Hydrometry,Rainfall,Depth_to_Groundwater_log,Depth_to_Groundwater_diff_1,Depth_to_Groundwater_diff_2
0,2009-01-01,-31.048571,1.657143,-28164.918857,2.371429,0.0,3.435553,0.000000,0.000000
1,2009-01-08,-30.784286,4.571429,-29755.789714,2.314286,2.0,3.427004,0.264286,0.264286
2,2009-01-15,-30.420000,7.528571,-25463.190857,2.300000,0.2,3.415100,0.364286,0.100000
3,2009-01-22,-30.018571,6.214286,-23854.422857,2.500000,4.1,3.401816,0.401429,0.037143
4,2009-01-29,-29.790000,5.771429,-25210.532571,2.500000,9.9,3.394173,0.228571,-0.172857


In [3]:
df.columns

Index(['Date', 'Depth_to_Groundwater', 'Temperature', 'Drainage_Volume',
       'River_Hydrometry', 'Rainfall', 'Depth_to_Groundwater_log',
       'Depth_to_Groundwater_diff_1', 'Depth_to_Groundwater_diff_2'],
      dtype='object')